In [445]:
import numpy as np
import pandas as pd
from pulp import *
from IPython.display import Image, display_png

In [468]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)
t = t.T.drop([
    'C00001', # H2O
    'C00080', # H+
    'C00009', # Phosphate
#     'C00008', # ADP
    'C00138', # Reduced ferredoxin
    'C00139', # Oxidized ferredoxin
    'C00002', # ATP
    'C00003', # NAD
    'C00004', # NADH
    'C00005',
    'C00006', 
    'C00010', 
    'C00011',
#     'C00111',
    'C00033',
#     'C00036',
    'C15602',
    'C15603'
]).T

In [469]:
h = [
    'R01786',
    'R02740',
    'R04779',
    'R01070',
    'R01061',
    'R01512',
    'R01518',
    'R00658',
    'R00200',
    'R01015',
    'R01196',
    'R00351',
    'R01325',
    'R01900',
    'R01899',
    'R00268',
    'R01197',
    'R10343',
    'R00341',
    'R02164',
    'R01082',
    'R00342',    
    'R00361',  
]

In [470]:
p = t.T[h].T

In [471]:
for i in p:
    if sum(p[i] != 0) == 0:
        p=p.drop(i, axis=1)

In [472]:
# g  = []
# for i in p.columns:
#     if sum(p[i] != 0.0) == 1:
#         g.append(i)
# g

In [473]:
# p = p.drop(g, axis=1)

In [474]:
x = [LpVariable(i) for i in p.columns]

In [475]:
f = pd.DataFrame([i for i in p.columns])
# (f[f[0] == 'C00024'].index)[0]

In [476]:
m = LpProblem(sense=LpMaximize)

m += x[(f[f[0] == 'C00091'].index)[0]]

In [477]:
capacity_plus = 100
capacity_minus = -100

for i in p.index:
    m += np.dot(list(p.T[i]), x) <= capacity_plus
    m += np.dot(list(p.T[i]), x) >= capacity_minus
    
m += x[(f[f[0] == 'C00267'].index)[0]] <= capacity_plus

In [478]:
pulp.LpStatus[m.solve()]

'Optimal'

In [479]:
m

NoName:
MAXIMIZE
1*C00091 + 0
SUBJECT TO
_C1: C00008 - C00267 + C00668 <= 100

_C2: C00008 - C00267 + C00668 >= -100

_C3: - C00668 + C05345 <= 100

_C4: - C00668 + C05345 >= -100

_C5: C00008 - C05345 + C05378 <= 100

_C6: C00008 - C05345 + C05378 >= -100

_C7: C00111 + C00118 - C05378 <= 100

_C8: C00111 + C00118 - C05378 >= -100

_C9: - C00118 + C00236 <= 100

_C10: - C00118 + C00236 >= -100

_C11: C00008 - C00197 + C00236 <= 100

_C12: C00008 - C00197 + C00236 >= -100

_C13: C00197 - C00631 <= 100

_C14: C00197 - C00631 >= -100

_C15: C00074 - C00631 <= 100

_C16: C00074 - C00631 >= -100

_C17: C00008 - C00022 + C00074 <= 100

_C18: C00008 - C00022 + C00074 >= -100

_C19: C00111 - C00118 <= 100

_C20: C00111 - C00118 >= -100

_C21: C00022 - C00024 <= 100

_C22: C00022 - C00024 >= -100

_C23: C00024 + C00036 - C00158 <= 100

_C24: C00024 + C00036 - C00158 >= -100

_C25: - C00158 + C00417 <= 100

_C26: - C00158 + C00417 >= -100

_C27: - C00311 + C00417 <= 100

_C28: - C00311 + C00417

In [480]:
m.status

1